In [2]:
import tensorflow as tf

In [4]:
# tf.enable_eager_execution()
my_graph = tf.Graph()


In [5]:
with my_graph.as_default():
    a = tf.placeholder(tf.float32)
    b = tf.placeholder(tf.float32)
    c = tf.constant([10.0, 20.0])
    total = a+b+c

    with tf.Session(graph=my_graph) as sess:
        print(sess.run(total, feed_dict={a:[3,1], b:[1,3], c:[100,200]}))


[104. 204.]


In [0]:
with my_graph.as_default():
    arr1 = [x*x for x in range(10)]
    arr2 = [x*x*x for x in range(10)]
    slices = tf.data.Dataset.from_tensor_slices((arr1, arr2))
    print(slices.output_shapes)
    slices = slices.batch(2).repeat(1)
    print(slices)
    next_item = slices.make_one_shot_iterator().get_next()
    print(next_item)

    with tf.Session(graph=my_graph) as sess:
        while True:
            try:
                print(sess.run(next_item))
            except tf.errors.OutOfRangeError:
                break

In [0]:
with my_graph.as_default():
    r = tf.random_normal([10,3])
    print(r)
    slices = tf.data.Dataset.from_tensor_slices(r)
    print(slices.output_shapes)
    iterator = slices.make_initializable_iterator()
    next_item = iterator.get_next()
    
    with tf.Session(graph=my_graph) as sess:
        sess.run(iterator.initializer)
        while True:
            try:
                print(sess.run(next_item))
            except tf.errors.OutOfRangeError:
                break

In [0]:
with my_graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    linear_model = tf.layers.Dense(units=2)
    y = linear_model(x)
    print(y.shape)
    
    init = tf.global_variables_initializer()
    
    with tf.Session(graph=my_graph) as sess:
        sess.run(init)
        print(sess.run(y, feed_dict={x:[[1, 2, 3], [4, 5, 6]]}))
        print(linear_model.weights)

In [0]:
with my_graph.as_default():
    sales = [[5,10], [6,12], [2,4], [10,20]]
    department = ["sports", "gardening", "toys", "sports"]
    features = {"sales":sales, "department":department}
    department_column = tf.feature_column.categorical_column_with_vocabulary_list("department", ["sports", "gardening", "toys"])
    department_column = tf.feature_column.indicator_column(department_column)
    columns = [tf.feature_column.numeric_column("sales", shape=[2]),
              department_column]
    inputs = tf.feature_column.input_layer(features, columns)
    
    var_init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    
    with tf.Session(graph=my_graph) as sess:
        sess.run((var_init, table_init))
        print(sess.run(inputs))

In [95]:
with my_graph.as_default():
    x = tf.constant([[1],[2],[3],[4]], dtype=tf.float32)
    y_true = tf.constant([[0],[-1],[-2],[-3]], dtype=tf.float32)
    
#     ds = tf.data.Dataset.from_tensor_slices((x, y_true))
    
    linear_model = tf.layers.Dense(units=1)
    y_predicted = linear_model(x)
    print(linear_model.weights)
    loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_predicted)
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)
    var_init = tf.global_variables_initializer()
    
    with tf.Session(graph=my_graph) as sess:
        sess.run(var_init)
        for i in range(100):
            _, loss_val = sess.run((train, loss))
            if i%10 == 0:
                print(loss_val)
        print("Predicted output:")
        print(sess.run(y_predicted))
        print("Final loss: ", loss_val)
        print(sess.run(linear_model.weights))

[<tf.Variable 'dense_64/kernel:0' shape=(1, 1) dtype=float32_ref>, <tf.Variable 'dense_64/bias:0' shape=(1,) dtype=float32_ref>]
14.229039
0.67458
0.30605727
0.27972442
0.26322266
0.24789605
0.23346704
0.219878
0.20707999
0.19502683
Predicted output:
[[-0.6920138]
 [-1.3353281]
 [-1.9786423]
 [-2.6219566]]
Final loss:  0.18478003
[array([[-0.64331424]], dtype=float32), array([-0.04869957], dtype=float32)]
